In [1]:
import numpy as np

In [2]:
# Загрузить и подготовить тренировочные данные из формата CSV в список
training_data = open("dataset/Data_train.csv", 'r') # 'r' - открываем файл для чтения
training_data_list = training_data.readlines() # readlines() - читает все строки в файле в переменную training_data_list
training_data.close() # закрываем файл csv

# Загрузить и подготовить тестовые данные из формата CSV в список
test_data = open("dataset/Data_test.csv", 'r') # 'r' - открываем файл для чтения
test_data_list = test_data.readlines()# Загрузить и подготовить тестовые данные из формата CSV в список 
test_data.close() # закрываем файл csv

In [3]:
# Определение класса нейронной сети
class neuron_Net:
    
    # Инициализация весов нейронной сети
    def __init__(self, input_num, neuron_num, learningrate): #констр.(кол-во входов, кол-во нейронов)
                                         # МАТРИЦА ВЕСОВ
        # Задаем матрицу весов как случайное от -0,5 до 0,5
        self.weights = (np.random.rand(neuron_num, input_num) -0.5) 
        # Задаем параметр скорости обучения
        self.lr = learningrate
        
        pass
    
    # Метод обучения нейронной сети
    def train(self, inputs_list, targets_list): # принимает (вх. список данных, ответы)
        # Преобразовать список входов в вертикальный массив. .T - транспонирование
        inputs_x = np.array(inputs_list, ndmin=2).T # матрица числа
        targets_Y = np.array(targets_list, ndmin=2).T # матрица ответов: какое это число
        
                                           # ВЫЧИСЛЕНИЕ СИГНАЛОВ
        # Вычислить сигналы в нейронах. Взвешенная сумма.
        x = np.dot(self.weights, inputs_x) # dot - умножение матриц X = W*I = weights * inputs
        # Вычислить сигналы, выходящие из нейрона. Функция активации - сигмоида(x)
        y = 1/(1+np.exp(-x))        
        
                                            # ВЫЧИСЛЕНИЕ ОШИБКИ
        #  Ошибка E = -(цель - фактическое значение) 
        E = -(targets_Y - y) 
        
                                            # ОБНОВЛЕНИЕ ВЕСОВ
        # Меняем веса по каждой связи
        self.weights -= self.lr * np.dot((E * y * (1.0 - y)), np.transpose(inputs_x))
        
        pass
    
    # Метод прогона тестовых значений
    def query(self, inputs_list): # Принимает свой набор тестовых данных
        # Преобразовать список входов в вертикальный 2D массив. 
        inputs_x = np.array(inputs_list, ndmin=2).T 
        
        # Вычислить сигналы в нейронах. Взвешенная сумма.
        x = np.dot(self.weights, inputs_x)
        # Вычислить сигналы, выходящие из нейрона. Сигмоида(x)
        y = 1/(1+np.exp(-x))
        
        return y

In [4]:
                            # ЗАДАЁМ ПАРАМЕТРЫ СЕТИ
# Количество входных данных, нейронов
data_input = 15
data_neuron = 2

# Cкорость обучения
learningrate = 0.1

# Создать экземпляр нейронной сети
n = neuron_Net(data_input, data_neuron, learningrate)

In [5]:
                            # ОБУЧЕНИЕ
# Зададим количество эпох
epochs = 40000
# Прогон по обучающей выборке
for e in range(epochs):
    for i in training_data_list:
        # Получить входные данные числа
        all_values = i.split(',') # split(',') - раздел строку на символы где запятая "," символ разделения
        inputs_x = np.asfarray(all_values[1:])
        
        # Получить целевое значение Y, (ответ - какое это число)
        targets_Y = int(all_values[0])  # перевод символов в int, 0 элемент - ответ
        
        # создать целевые выходные значения (все 0.01, кроме нужной метки, которая равна 0.99)
        targets_Y = np.zeros(data_neuron) + 0.01
        
        # Получить целевое значение Y, (ответ - какое это число). all_values[0] - целевая метка для этой записи
        if int(all_values[0]) <= 1: # цель <= 1 потому как распознаём только 2 числа, 0 и 1.
            targets_Y[int(all_values[0])] = 0.99
            
        n.train(inputs_x, targets_Y) # наш метод train - обучение нейронной сети                      

In [6]:
# Вывод обученных весов
print('Весовые коэффициенты:\n', n.weights)

Весовые коэффициенты:
 [[ 0.55376239 -0.2659114  -0.55801795  1.63794241 -0.39180754  0.18253672
  -1.86935497 -4.51957151 -1.10400713  3.68316632 -1.31690796  0.48928634
   0.63951451  0.6593403  -3.74255197]
 [-4.18510101 -3.35072555 -0.15142484 -0.89959198 -0.54914752  1.80364436
  -2.10131379 -0.79061126  0.94477939  0.27198919 -0.42900452  0.13232677
  -0.54182952  0.16586589  1.24938759]]


In [7]:
# Еще раз пройдем по обучающей выборке
for i in training_data_list:
    all_values = i.split(',') # split(',') - раздел строку на символы где запятая "," символ разделения
    inputs_x = np.asfarray(all_values[1:])
    # Прогон по сети
    outputs = n.query(inputs_x)
    print(i[0], 'Вероятность:\n', outputs)

0 Вероятность:
 [[ 0.98282717]
 [ 0.0018778 ]]
1 Вероятность:
 [[ 0.01110791]
 [ 0.98279182]]
2 Вероятность:
 [[ 0.01280932]
 [ 0.00169931]]
3 Вероятность:
 [[  4.97172575e-05]
 [  2.29715820e-03]]
4 Вероятность:
 [[ 0.00010462]
 [ 0.0130299 ]]
5 Вероятность:
 [[ 0.00018235]
 [ 0.00010789]]
6 Вероятность:
 [[ 0.01247943]
 [ 0.00070262]]
7 Вероятность:
 [[ 0.01036096]
 [ 0.01685072]]
8 Вероятность:
 [[ 0.00498377]
 [ 0.00085023]]
9 Вероятность:
 [[ 0.01543393]
 [ 0.00064968]]


In [8]:
# Если вероятность больше 0,5 и номер выхода совпадает с ответом, то считаем что сеть, 
#на своем определенном выходе, узнала цифру. 
for i in training_data_list:
    all_values = i.split(',') # split(',') - раздел строку на символы где запятая "," символ разделения
    inputs_x = np.asfarray(all_values[1:])
    # Прогон по сети
    outputs = n.query(inputs_x)
    # индекс самого высокого значения соответствует метке
    label = np.argmax(outputs)
    if outputs[label]>0.5 and int(all_values[0]) == label:
        print(i[0], 'Узнал?: ', 'Да!')
    else:
        print(i[0], 'Узнал?: ', 'Нет!')                                         

0 Узнал?:  Да!
1 Узнал?:  Да!
2 Узнал?:  Нет!
3 Узнал?:  Нет!
4 Узнал?:  Нет!
5 Узнал?:  Нет!
6 Узнал?:  Нет!
7 Узнал?:  Нет!
8 Узнал?:  Нет!
9 Узнал?:  Нет!


In [9]:
# Проход по тестовой выборке
t = 0 # Счетчик номера нуля тестовой выборки
t1 = 0 # Счетчик номера единицы тестовой выборки
for i in test_data_list:
    all_values = i.split(',') # split(',') - раздел строку на символы где запятая "," символ разделения
    inputs_x = np.asfarray(all_values[1:])
    t += 1
    # Прогон по сети
    outputs = n.query(inputs_x)
    # индекс самого высокого значения соответствует метке
    label = np.argmax(outputs)
    if t <= 6:
        print('Вероятность что узнал 0 -',t, '?', outputs[label])
    else:
        t1 += 1
        print('Вероятность что узнал 1 -',t1, '?', outputs[label])

Вероятность что узнал 0 - 1 ? [ 0.95590294]
Вероятность что узнал 0 - 2 ? [ 0.98378171]
Вероятность что узнал 0 - 3 ? [ 0.63522571]
Вероятность что узнал 0 - 4 ? [ 0.92786908]
Вероятность что узнал 0 - 5 ? [ 0.78988343]
Вероятность что узнал 0 - 6 ? [ 0.76715129]
Вероятность что узнал 1 - 1 ? [ 0.98163393]
Вероятность что узнал 1 - 2 ? [ 0.98125632]
Вероятность что узнал 1 - 3 ? [ 0.94877843]
Вероятность что узнал 1 - 4 ? [ 0.95536855]
Вероятность что узнал 1 - 5 ? [ 0.9817356]
Вероятность что узнал 1 - 6 ? [ 0.95543842]


In [10]:
t = 0 # Счетчик номера нуля тестовой выборки
t1 = 0 # Счетчик номера единицы тестовой выборки
# Если вероятность больше 0,5 и номер выхода совпадает с ответом, то считаем что сеть, 
#на своем определенном выходе, узнала цифру. 
for i in test_data_list:
    all_values = i.split(',') # split(',') - раздел строку на символы где запятая "," символ разделения
    inputs_x = np.asfarray(all_values[1:])
    # Прогон по сети
    outputs = n.query(inputs_x)
    # индекс самого высокого значения соответствует метке
    label = np.argmax(outputs)
    t += 1
    if outputs[label]>0.5 and int(all_values[0]) == label:
        print(i[0], 'Узнал?:',t, 'Да!')
    else:
        t1 += 1
        print(i[0], 'Узнал?:',t1, 'Нет!')

0 Узнал?: 1 Да!
0 Узнал?: 2 Да!
0 Узнал?: 3 Да!
0 Узнал?: 4 Да!
0 Узнал?: 5 Да!
0 Узнал?: 6 Да!
1 Узнал?: 7 Да!
1 Узнал?: 8 Да!
1 Узнал?: 9 Да!
1 Узнал?: 10 Да!
1 Узнал?: 11 Да!
1 Узнал?: 12 Да!
